In [ ]:
%pip install -U scikit-learn

In [ ]:
%pip install xgboost

In [ ]:
%pip install sktime --ignore-installed llvmlite

In [ ]:
%pip install -U scikit-optimize

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [9]:
# Read CSV file into dataframe
us_rates_rec_df = pd.read_csv('us_rates_rec.csv')

# Convert `date` column to monthly periods
us_rates_rec_df['date'] = pd.to_datetime(us_rates_rec_df['date'], format = '%Y-%m-%d')
us_rates_rec_df['month'] = us_rates_rec_df['date'].dt.to_period('M')

us_rates_rec_df.shape

(2014, 15)

In [10]:
# Set start and end dates in monthly periods

date_series = pd.date_range(start='1953-04-01', end='2000-12-31')
date_series = date_series.to_period('M')

In [11]:
# Define variables used in models

max_lags = 18
max_horizon = 12
time_start = date_series[0] # + max_lags
time_end = date_series[-1]
model_vars = ['spr1', 'spr5', 'spr10', 'hyspr']

In [12]:
# Filter for data starting from 1953m4

mask_start = us_rates_rec_df['month'] >= time_start
mask_end = us_rates_rec_df['month'] > time_end

us_rates_rec_df = us_rates_rec_df[mask_start].reset_index(drop=True)
us_rates_rec_df.shape

(834, 15)

In [ ]:
# Naive Model 1 Accuracy: Always predict expansion (0)

naive_accuracy = {}

naive_accuracy['Full Dataset'] = 1 - us_rates_rec_df['rec'].sum()/us_rates_rec_df.shape[0]
naive_accuracy['Train Dataset'] = 1 - us_rates_rec_df.loc[~mask_end, 'rec'].sum()/us_rates_rec_df.loc[~mask_end].shape[0]
naive_accuracy['Test Dataset'] = 1 - us_rates_rec_df.loc[mask_end, 'rec'].sum()/us_rates_rec_df.loc[mask_end].shape[0]

pprint(naive_accuracy)

{'Full Dataset': 0.8645083932853717,
 'Test Dataset': 0.89272030651341,
 'Train Dataset': 0.8516579406631762}


In [13]:
#Create lagged variables for rec and ir spreads

for lag in range(1, max_lags + max_horizon):
    if lag == 12:
        model_vars += ['rec']
    lagged = us_rates_rec_df[model_vars].shift(lag)
    lagged.columns = [x + 'L' + str(lag) for x in model_vars]

    us_rates_rec_df = pd.concat((us_rates_rec_df, lagged), axis=1)

model_vars.remove('rec')

In [ ]:
# Correlation matrix where absolute correlation coefficient > 0.8

corr_matrix = us_rates_rec_df.corr()
corr_matrix.where(corr_matrix.abs().gt(0.8)).stack()[:23]

rec    rec        1.000000
       recL1      0.887377
spr1   spr1       1.000000
       spr1L1     0.901540
spr5   spr5       1.000000
       spr10      0.953546
       spr5L1     0.944415
       spr5L2     0.866180
       spr5L3     0.801678
       spr10L1    0.908335
       spr10L2    0.840818
spr10  spr5       0.953546
       spr10      1.000000
       spr5L1     0.906134
       spr5L2     0.839114
       spr10L1    0.958322
       spr10L2    0.896323
       spr10L3    0.844189
hyspr  hyspr      1.000000
       hysprL1    0.968097
       hysprL2    0.917090
       hysprL3    0.870418
       hysprL4    0.827970
dtype: float64

In [14]:
# Naive Model 2 Accuracy: rec = recL1

from sklearn.metrics import accuracy_score

naive_accuracy2 = {}

mask_start = us_rates_rec_df['month'] >= time_start
mask_end = us_rates_rec_df['month'] > time_end

naive_pred_full = us_rates_rec_df['recL1'][1:]
naive_pred_train = us_rates_rec_df.loc[~mask_end, 'recL1'][1:]
naive_pred_test = us_rates_rec_df.loc[mask_end, 'recL1'][1:]

naive_accuracy2['Full Dataset'] = accuracy_score(us_rates_rec_df['rec'][1:], naive_pred_full)
naive_accuracy2['Train Dataset'] = accuracy_score(us_rates_rec_df.loc[~mask_end, 'rec'][1:], naive_pred_train)
naive_accuracy2['Test Dataset'] = accuracy_score(us_rates_rec_df.loc[mask_end, 'rec'][1:], naive_pred_test)

pprint(naive_accuracy2)

{'Full Dataset': 0.9735894357743097,
 'Test Dataset': 0.9769230769230769,
 'Train Dataset': 0.972027972027972}


In [15]:
# Select only columns to be used in models

us_rates_rec_df = us_rates_rec_df[['month', 'rec', 'spr1', 'spr5', 'spr10', 'hyspr'] 
    + [f'recL{lag}' for lag in range(12, max_lags+max_horizon)] + [f'{x}L{lag}' for x in model_vars for lag in range(1, max_lags+max_horizon)]]

us_rates_rec_df.set_index('month', inplace = True)

us_rates_rec_df.shape

(834, 139)

In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split
from skopt.space import Real, Integer
from sklearn import metrics

# Split X and y variables into sequential train and test sets
# This means that the last 261 observations constitute the test set
X_train, X_test = temporal_train_test_split(us_rates_rec_df.drop('rec', axis=1), test_size=261)
y_train, y_test = temporal_train_test_split(us_rates_rec_df['rec'], test_size=261)

# Define the parameter grid for cross-validation to search through
param_grid = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(2, 6),
    'gamma': Real(0, 1.0),
    'min_child_weight': Real(0, 10.0),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.4, 1.0),
    'reg_lambda': Real(0, 100),
    'scale_pos_weight': Real(1, 6)
}

# Define (negative) log-loss as the loss function, aka scorer
log_loss = metrics.make_scorer(metrics.log_loss, greater_is_better=False, needs_proba=True, labels=[0, 1])

print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (573, 138), X_test: (261, 138), y_train: (573,), y_test: (261,)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, average_precision_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score
from sktime.utils.plotting import plot_series
import seaborn as sns

# Create empty dataframes and dictionaries to store results
xgb_train_df = pd.DataFrame(
    columns=['Obs', 'TN', 'FN', 'FP', 'TP', 'Accuracy', 'F1 Score', 'PR AUC', 'Precision', 'Sensitivity', 'Specificity', 'ROC AUC']
).reindex([f'h{h}' for h in range(1, max_horizon+1)])

xgb_test_df = pd.DataFrame(
    columns=['Obs', 'TN', 'FN', 'FP', 'TP', 'Accuracy', 'F1 Score', 'PR AUC', 'Precision', 'Sensitivity', 'Specificity', 'ROC AUC']
).reindex([f'h{h}' for h in range(1, max_horizon+1)])

xgb_pred_df = pd.DataFrame().reindex(us_rates_rec_df.index)
xgb_prob_df = pd.DataFrame().reindex(us_rates_rec_df.index)

xgb_params = {}

xgb_features = {}

# Loop through each forecast horizon (in months) from 1 to 12
for h in range(1, max_horizon+1):
    # Create new filters for each forecast horizon, eg. start date increases by 1 period
    time_start2 = time_start + h - 1
    p2 = max_lags + h
    mask_startX = X_train.index >= time_start2
    mask_starty = y_train.index >= time_start2

    # Select only X and y columns and observations for the relevant forecast horizon
    X_train_h = X_train.loc[mask_startX, [f'recL{lag}' for lag in range(12, p2)] + [f'{x}L{lag}' for x in model_vars for lag in range(h, p2)]]
    X_test_h = X_test[[f'recL{lag}' for lag in range(12, p2)] + [f'{x}L{lag}' for x in model_vars for lag in range(h, p2)]]
    y_train_h = y_train.loc[mask_starty]

    # Create an expanding window splitter for cross-validation
    cv = TimeSeriesSplit(n_splits=5)

    # Create an XGB classifier with a fixed learning rate and objective
    # Set the random state to 42 (the meaning of life, the universe, and everything!) for reproducible results
    xgb_clf = XGBClassifier(learning_rate=0.01, objective='binary:logistic', random_state=42)

    # Create a Bayesian optimization search object with search spaces, scoring and cv as previously defined
    # Fit the search object on the train dataset for the respective forecast horizon
    xgb_search = BayesSearchCV(estimator=xgb_clf, search_spaces=param_grid, n_iter=100, scoring=log_loss, cv=cv, random_state=42)
    xgb_search.fit(X=X_train_h, y=y_train_h)

    # Extract the best parameters found
    # Fit an XGB classifier with the best parameters on the train set
    xgb_params[f'h{h}'] = xgb_search.best_params_
    xgb_fitted = XGBClassifier(**xgb_search.best_estimator_.get_params()).fit(X=X_train_h, y=y_train_h)

    # Calculate the predicted recession binary values and probabilities for the train set
    y_pred_train = xgb_fitted.predict(X=X_train_h)
    y_prob_train = xgb_fitted.predict_proba(X_train_h)[:, 1]

    # Unravel the confusion matrix to obtain TN, FP, FN and TP values for the train set
    tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train_h, y_pred_train).ravel()

    xgb_train_df.loc[f'h{h}', 'Obs'] = len(X_train_h)
    xgb_train_df.loc[f'h{h}', 'TN'] = tn_train
    xgb_train_df.loc[f'h{h}', 'FN'] = fn_train
    xgb_train_df.loc[f'h{h}', 'FP'] = fp_train
    xgb_train_df.loc[f'h{h}', 'TP'] = tp_train

    # Compute the 7 different evaluation metrics for fitted model on the train set
    xgb_train_df.loc[f'h{h}', 'Accuracy'] = accuracy_score(y_train_h, y_pred_train)
    xgb_train_df.loc[f'h{h}', 'F1 Score'] = f1_score(y_train_h, y_pred_train)
    xgb_train_df.loc[f'h{h}', 'PR AUC'] = average_precision_score(y_train_h, y_prob_train)
    xgb_train_df.loc[f'h{h}', 'Precision'] = precision_score(y_train_h, y_pred_train)
    xgb_train_df.loc[f'h{h}', 'Sensitivity'] = recall_score(y_train_h, y_pred_train)
    xgb_train_df.loc[f'h{h}', 'Specificity'] = recall_score(y_train_h, y_pred_train, pos_label=0)
    xgb_train_df.loc[f'h{h}', 'ROC AUC'] = roc_auc_score(y_train_h, y_prob_train)

    # Calculate the predicted recession binary values and probabilities for the test set
    y_pred_test = xgb_fitted.predict(X=X_test_h)
    y_prob_test = xgb_fitted.predict_proba(X_test_h)[:, 1]
    
    # Unravel the confusion matrix to obtain TN, FP, FN and TP values for the test set
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()

    xgb_test_df.loc[f'h{h}', 'Obs'] = len(X_test_h)
    xgb_test_df.loc[f'h{h}', 'TN'] = tn_test
    xgb_test_df.loc[f'h{h}', 'FN'] = fn_test
    xgb_test_df.loc[f'h{h}', 'FP'] = fp_test
    xgb_test_df.loc[f'h{h}', 'TP'] = tp_test

    # Compute the 7 different evaluation metrics for fitted model on the test set
    xgb_test_df.loc[f'h{h}', 'Accuracy'] = accuracy_score(y_test, y_pred_test)
    xgb_test_df.loc[f'h{h}', 'F1 Score'] = f1_score(y_test, y_pred_test)
    xgb_test_df.loc[f'h{h}', 'PR AUC'] = average_precision_score(y_test, y_prob_test)
    xgb_test_df.loc[f'h{h}', 'Precision'] = precision_score(y_test, y_pred_test)
    xgb_test_df.loc[f'h{h}', 'Sensitivity'] = recall_score(y_test, y_pred_test)
    xgb_test_df.loc[f'h{h}', 'Specificity'] = recall_score(y_test, y_pred_test, pos_label=0)
    xgb_test_df.loc[f'h{h}', 'ROC AUC'] = roc_auc_score(y_test, y_prob_test)

    # Extract the top 20 features by importance
    feature_importance = pd.Series(xgb_fitted.feature_importances_, index=X_train_h.columns)
    best_features = feature_importance.sort_values(ascending=False).head(20)
    xgb_features[f'h{h}'] = best_features

    # Convert np arrays to datetime series for time series plotting
    y_pred = pd.concat([pd.Series(y_pred_train).set_axis(y_train_h.index), pd.Series(y_pred_test).set_axis(y_test.index)])
    y_prob = pd.concat([pd.Series(y_prob_train).set_axis(y_train_h.index), pd.Series(y_prob_test).set_axis(y_test.index)])

    xgb_pred_df[f'h{h}'] = y_pred
    xgb_prob_df[f'h{h}'] = y_prob

    # Plot the actual recession binary values with the predicted binary values
    plot_series(y_train_h, y_test, y_pred, labels=['y_train', 'y_test', 'y_pred'], x_label='Date', y_label='Rec(Binary)')
    plt.title(f'us_recbinary_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'us_recbinary_xgb_h{h}.png')
    plt.close()

    # Plot the actual recession binary values with the predicted probabilities
    plot_series(y_train_h, y_test, y_prob, labels=['y_train', 'y_test', 'y_prob'], x_label='Date', y_label='Pr(Rec)')
    plt.title(f'us_recprob_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'us_recprob_xgb_h{h}.png')
    plt.close()

    # Plot the top 20 features by importance
    # Save all three graphs and close them for matplotlib to release memory
    plt.figure(figsize=(20,28))
    sns.barplot(x=best_features.values, y=best_features.index)
    plt.title(f'feature_importance_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'feature_importance_xgb_h{h}.png')
    plt.close()

In [ ]:
# Export results and estimates to CSV files

xgb_train_df.to_csv('rec_xgb_train_results.csv')
xgb_test_df.to_csv('rec_xgb_test_results.csv')
xgb_pred_df.to_csv('rec_xgb_pred_estimates.csv')
xgb_prob_df.to_csv('rec_xgb_prob_estimates.csv')

In [ ]:
# Perform some manipulation and export feature importances to CSV

tmp_feature = xgb_features['h1'].rename_axis('h1_feature').reset_index(name='h1_importance')
feature_importance_df = tmp_feature.T

for h in range(2, max_horizon+1):
    tmp_feature = xgb_features[f'h{h}'].rename_axis(f'h{h}_feature').reset_index(name=f'h{h}_importance')
    feature_importance_df = pd.concat([feature_importance_df, tmp_feature.T])

feature_importance_df.to_csv('rec_xgb_feature_importances.csv')

In [ ]:
# Perform some manipulation and export best parameters to CSV
# This can be directly imported and used to reproduce results in future

xgb_params_df = pd.DataFrame.from_dict(xgb_params, orient='index')
xgb_params_df = xgb_params_df.reindex([f'h{h}' for h in range(1, max_horizon+1)])

xgb_params_df.to_csv('rec_xgb_parameters.csv')

,accuracy,subsample,scale_pos_weight,n_estimators,min_child_weight,max_depth,gamma,colsample_bytree
h1,0.888889,0.8,1,100,10.0,5,0.4,0.6
h2,0.873563,0.7,1,100,10.0,6,0.6,0.7
h3,0.885057,0.8,1,100,10.0,5,0.4,0.6
h4,0.881226,0.7,1,100,10.0,6,0.6,0.7
h5,0.873563,0.8,1,100,10.0,5,0.4,0.6
h6,0.877395,0.7,1,100,10.0,6,0.6,0.7
h7,0.885057,0.7,1,100,10.0,6,0.6,0.7
h8,0.881226,0.7,1,100,10.0,6,0.6,0.7
h9,0.885057,0.7,1,100,10.0,6,0.6,0.7
h10,0.869732,0.7,1,100,7.0,2,0.4,0.4
